In [ ]:
# Get max timestamps from previous Extract_Max_ts task
max_match_ts = dbutils.jobs.taskValues.get(
    taskKey="Extract_Max_ts", 
    key="max_match_ts", 
    debugValue="2026-01-01 00:00:00"
)

max_delivery_ts = dbutils.jobs.taskValues.get(
    taskKey="Extract_Max_ts", 
    key="max_delivery_ts", 
    debugValue="2026-01-01 00:00:00"
)

print("Max Match TS:", max_match_ts)
print("Max Delivery TS:", max_delivery_ts)


In [ ]:
match_df = spark.sql(f"""
    SELECT *
    FROM bronze.match_loading
    WHERE ingestion_ts <= TIMESTAMP('{max_match_ts}')
""")

archive_match_path = "/Volumes/ipl_data_engineering/bronze/ipl_archive/match_backup/"

# Save as CSV
(
    match_df
    .coalesce(1)  # optional: single file
    .write
    .mode("append")
    .option("header", "true")
    .csv(archive_match_path)
)

print(f"Archived match_loading to {archive_match_path}")


In [ ]:
delivery_df = spark.sql(f"""
    SELECT *
    FROM bronze.delivery_loading
    WHERE ingestion_ts <= TIMESTAMP('{max_delivery_ts}')
""")

archive_delivery_path = "/Volumes/ipl_data_engineering/bronze/ipl_archive/delivery_backup/"

(
    delivery_df
    .coalesce(1)
    .write
    .mode("append")
    .option("header", "true")
    .csv(archive_delivery_path)
)

print(f"Archived delivery_loading to {archive_delivery_path}")


In [ ]:
from datetime import datetime
from pyspark.sql import Row

audit_data = [Row(
    max_match_ts=max_match_ts,
    max_delivery_ts=max_delivery_ts,
    archived_at=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
)]

audit_df = spark.createDataFrame(audit_data)

audit_path = "/Volumes/ipl_data_engineering/bronze/ipl_archive/max_ts_snapshot/"

(
    audit_df
    .coalesce(1)
    .write
    .mode("append")
    .option("header", "true")
    .csv(audit_path)
)

print(f"Saved audit snapshot to {audit_path}")
